# 2.Data Cleaning and description
***

<small> Note: The 'utils.py' file contains **specific functions** according to the standards of Exploratory Data Analysis (EDA) and Descriptive Data Analysis (DDA). The functions with the prefix 'eda' are described in the 'utils.py' doc <small>

For this Eda, the main question to answer is the following:

 **It is possible to predict the obessity levels in the US with the sociograph census provided data?** 

In [15]:
# LIBRARIES
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import utils as eda

# DATASET
total_data = pd.read_csv(r'C:\Users\rnogu\OneDrive\Documentos\GitHub\Regularized-linear-regression\data\raw\raw_total_data.csv')
total_data.info()

target = 'Obesity_number'

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3140 entries, 0 to 3139
Columns: 108 entries, fips to Urban_rural_code
dtypes: float64(61), int64(45), object(2)
memory usage: 2.6+ MB


The dataset has a total of 3140 rows and 108 columns.

1. Drop duplicates

In [16]:
total_data = total_data.drop_duplicates().reset_index(drop = True)
total_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3140 entries, 0 to 3139
Columns: 108 entries, fips to Urban_rural_code
dtypes: float64(61), int64(45), object(2)
memory usage: 2.6+ MB


The dataset has no duplicate rows.

2. Filter the variables with strongest correlation.

Due to the ammount of predictors, the feature selection step will be done now and not at the end.

2.1. Standard value scalation

In [17]:
from sklearn.preprocessing import StandardScaler

# List of every column type
aux_type = total_data.dtypes

# Create a list with numeric types for scalation
num_columns = [col for col in list(aux_type[aux_type != "object"].index) if col != target]

# Create and fit the scaler
scaler = StandardScaler()
norm_features = scaler.fit_transform(total_data[num_columns])

# Create the new dataset 
scal_total_data = pd.DataFrame(norm_features, index= total_data.index, columns= num_columns)

# add the chosen target
scal_total_data[target] = total_data[target]
scal_total_data.head(2)

,fips,TOT_POP,0-9,0-9 y/o % of total pop,19-Oct,10-19 y/o % of total pop,20-29,20-29 y/o % of total pop,30-39,30-39 y/o % of total pop,...,diabetes_prevalence,diabetes_Lower 95% CI,diabetes_Upper 95% CI,diabetes_number,CKD_prevalence,CKD_Lower 95% CI,CKD_Upper 95% CI,CKD_number,Urban_rural_code,Obesity_number
0,-1.940874,-0.145679,-0.142421,0.158006,-0.135556,0.573496,-0.153144,0.027610,-0.139384,0.588469,...,-0.063696,-0.07172,-0.089834,-0.129902,-0.609615,-0.582796,-0.669652,-0.147523,-1.082865,15193
1,-1.940742,0.341296,0.287476,-0.242861,0.320383,-0.193107,0.183774,-0.469965,0.230620,-0.110300,...,-0.394103,-0.41490,-0.337677,0.376251,-0.433549,-0.393279,-0.343373,0.389791,-0.420704,50761


2.2. Feature selection

For this model, 5 different scenarios will be performed, using top 20, 30, 40, 50 and 60 variables.

In [22]:
from sklearn.feature_selection import SelectKBest, f_regression

# Aux variables
kbest_options = [20, 30, 40, 50, 60]
kbest_results = {}

for options in kbest_options:
    # create a loop that will make top 20, top 30 ....
    selector = SelectKBest(score_func=f_regression, k=options)
    X_selected = selector.fit_transform(scal_total_data.drop(target, axis=1), scal_total_data[target])

    # create a dataframe for each top
    selected_features_df = pd.DataFrame(X_selected, columns=scal_total_data.drop(target, axis=1).columns[selector.get_support()])
    selected_features_df[target] = total_data[target]

    # Store the tops results in the dataframe
    kbest_results[options] = selected_features_df

kbest_results[20]

,TOT_POP,0-9,19-Oct,20-29,30-39,40-49,50-59,60-69,POP_ESTIMATE_2018,High school diploma only 2014-18,...,Civilian_labor_force_2018,Employed_2018,Total Population,county_pop2018_18 and older,anycondition_number,Heart disease_number,COPD_number,diabetes_number,CKD_number,Obesity_number
0,-0.145679,-0.142421,-0.135556,-0.153144,-0.139384,-0.122010,-0.138129,-0.176461,-0.145679,-0.132905,...,-0.151210,-0.151024,-0.143727,-0.148472,-0.133552,-0.144279,-0.138900,-0.129902,-0.147523,15193
1,0.341296,0.287476,0.320383,0.183774,0.230620,0.332513,0.382732,0.521999,0.341296,0.427106,...,0.251083,0.253105,0.308018,0.348569,0.358089,0.500641,0.563986,0.376251,0.389791,50761
2,-0.237785,-0.239429,-0.246181,-0.225971,-0.218759,-0.225725,-0.242545,-0.256141,-0.237785,-0.243746,...,-0.255403,-0.256470,-0.231549,-0.236483,-0.223123,-0.220242,-0.219763,-0.183415,-0.204321,8013
3,-0.245223,-0.246032,-0.254791,-0.230792,-0.220555,-0.226554,-0.246361,-0.271526,-0.245223,-0.224364,...,-0.253697,-0.254142,-0.242577,-0.243741,-0.244717,-0.260337,-0.256918,-0.229096,-0.242100,6894
4,-0.138966,-0.135053,-0.137140,-0.155888,-0.143570,-0.124495,-0.135648,-0.143459,-0.138966,-0.105947,...,-0.156845,-0.156502,-0.135714,-0.140695,-0.129476,-0.095857,-0.074198,-0.111247,-0.124105,15112
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3135,-0.183307,-0.158761,-0.166565,-0.183788,-0.149837,-0.173730,-0.196120,-0.202045,-0.183307,-0.185758,...,-0.179694,-0.180425,-0.175734,-0.189779,-0.208245,-0.239265,-0.237259,-0.218239,-0.212860,10672
3136,-0.243182,-0.247755,-0.264513,-0.232043,-0.200548,-0.217696,-0.250739,-0.261965,-0.243182,-0.326665,...,-0.214121,-0.213637,-0.241532,-0.239760,-0.291384,-0.295693,-0.311695,-0.267785,-0.261248,3655
3137,-0.251523,-0.230123,-0.240386,-0.248464,-0.228946,-0.241462,-0.262305,-0.271266,-0.251523,-0.269523,...,-0.252097,-0.252758,-0.248126,-0.256732,-0.275960,-0.292298,-0.296744,-0.263079,-0.263447,4944
3138,-0.288743,-0.284261,-0.289869,-0.277474,-0.265673,-0.275742,-0.293887,-0.312873,-0.288743,-0.337586,...,-0.280972,-0.281505,-0.286212,-0.289087,-0.314556,-0.326501,-0.338543,-0.290441,-0.292299,1730
